### Game Graph Class

In [2]:
class node:
    def __init__(self, value=None, left=None, right=None, parent=None):
        self.value=value
        self.left = left
        self.right = right
        self.parent = parent
        
def make_children(source):
    left = node(parent=source)
    right = node(parent=source)
    source.left = left
    source.right = right
    return left, right

def make_tree(n, leaf_values=None):
    if not leaf_values:
        leaf_values = [None]*pow(2,n)
    source = node()
    tree = [[] for  i in range(n+1)] #i is height
    tree[0].append(source)
    for i in range(n):
        for j in range(pow(2, i)):
            tree[i+1]+=make_children(tree[i][j])
            
    for j in range(pow(2, n)):
        tree[n][j].value = leaf_values[j]
    return tree



### Value calculation

In [14]:
# Example tree:
tree4 = make_tree(4)
import numpy as np
for v in tree4[-1]:
    v.value=np.random.random()
    
    
def DFS_visit(node, Paul=True):
    
    if not node.value:
        DFS_visit(node.left, Paul=not Paul)
        DFS_visit(node.right, Paul=not Paul)
        if node.left.value:
            if node.right.value:
                if Paul:
                    node.value = max(node.left.value, node.right.value)
                else:
                    node.value = min(node.left.value, node.right.value)
                    
# Example calculation:                    
DFS_visit(tree4[0][0])
tree4[0][0].value

0.6239178410815344

### Simulations

In [ ]:
even_trees = [make_tree(2*k) for k in np.random.randint(10, 20, 100)]
odd_trees = [make_tree(2*k + 1) for k in np.random.randint(10, 20, 100)]

def get_leaf_values(tree):
    for v in tree[-1]:
        v.value = np.random.random()

for tree in even_trees:
    get_leaf_values(tree)
    
for tree in odd_trees:
    get_leaf_values(tree)